In [1]:
library(tseries)

Warning message:
"package 'tseries' was built under R version 4.4.2"
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



In [2]:
data <- read.csv("./stationary_data.csv")
data

var1,var2,var3
<dbl>,<dbl>,<dbl>
8.778324,0.4549183,3.14660311
4.060246,5.1906868,3.67456524
-5.513257,1.9018173,-0.77639107
-4.536916,-3.6050020,-1.11632610
-3.135693,-2.4210588,-1.37984115
5.493270,-1.5790918,1.00799378
2.897723,3.5143362,0.23645938
2.583402,1.4308751,0.60122078
1.070094,1.3454019,0.18339480


In [3]:
adf_var1 <- adf.test(data$var1, alternative = "stationary")
adf_var2 <- adf.test(data$var2, alternative = "stationary")
adf_var3 <- adf.test(data$var3, alternative = "stationary")

adf_var1
adf_var2
adf_var3


	Augmented Dickey-Fuller Test

data:  data$var1
Dickey-Fuller = -3.6816, Lag order = 2, p-value = 0.04417
alternative hypothesis: stationary



	Augmented Dickey-Fuller Test

data:  data$var2
Dickey-Fuller = -3.8711, Lag order = 2, p-value = 0.03064
alternative hypothesis: stationary



	Augmented Dickey-Fuller Test

data:  data$var3
Dickey-Fuller = -4.0902, Lag order = 2, p-value = 0.02011
alternative hypothesis: stationary
